# 1. GPU训练30轮次

In [4]:
import torchvision
import torch
from torch import nn
from torch.utils.data import DataLoader

# ----------------------
# 1. 硬件配置
# ----------------------
# 语法：检查当前环境是否支持 NVIDIA 显卡 (CUDA)。
# 作用：如果有显卡，代码后续会将模型和数据搬运到显卡上加速训练；否则使用 CPU。
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")

# ----------------------
# 2. 定义神经网络模型
# ----------------------
# 语法：继承 nn.Module 类，这是 PyTorch 中构建所有网络的基类。
class Tudui(nn.Module):
    def __init__(self):
        super().__init__()        
        # 语法：nn.Sequential 是一个容器，数据会按照定义的顺序依次通过这些层。
        # 作用：避免写很多行 self.conv1, self.relu1... 让代码更整洁。
        self.model1 = nn.Sequential(
            # Layer 1: 卷积层
            # 语法：输入通道3(RGB图片)，输出32个特征图，卷积核5x5，步长1，填充2。
            # 作用：提取图像的初步特征（如边缘、纹理）。
            nn.Conv2d(3, 32, 5, 1, 2), 
            
            # Layer 2: 池化层
            # 语法：最大池化，窗口大小2x2。
            # 作用：下采样，将图片尺寸缩小一半（特征图变小），减少计算量，保留主要特征。
            nn.MaxPool2d(2),
            
            # Layer 3: 卷积层
            # 语法：输入接上一层的32，输出32。
            nn.Conv2d(32, 32, 5, 1, 2),
            
            # Layer 4: 池化层
            nn.MaxPool2d(2),
            
            # Layer 5: 卷积层
            # 语法：输出通道增加到64，提取更抽象的高级特征。
            nn.Conv2d(32, 64, 5, 1, 2),
            
            # Layer 6: 池化层
            nn.MaxPool2d(2),
            
            # Layer 7: 展平层
            # 语法：将多维的特征图 (Batch_size, 64, 4, 4) 展平成一维向量 (Batch_size, 64*4*4)。
            # 作用：卷积层输出是立体的，全连接层需要一维线性的输入，所以必须展平。
            nn.Flatten(),
            
            # Layer 8: 全连接层 (Linear)
            # 语法：输入节点数 1024 (64*4*4)，输出节点数 64。
            # 作用：将提取的特征进行线性组合。
            nn.Linear(64*4*4, 64),
            
            # Layer 9: 输出层
            # 语法：输入64，输出10。
            # 作用：CIFAR10 有10个分类，所以最后输出必须是 10 个数字（代表10个类别的概率/分数）。
            nn.Linear(64, 10)
        )
        
    # 语法：前向传播函数，必须重写。
    # 作用：定义数据 x 进入网络后怎么走。这里直接通过 model1 容器即可。
    def forward(self, x):
        x = self.model1(x)
        return x

# ----------------------
# 3. 准备数据
# ----------------------
# 语法：下载并加载 CIFAR10 训练集。
# 参数 root: 保存路径；train=True: 下载训练集；transform: 将图片转换为 Tensor 格式（归一化到 0-1）。
train_data = torchvision.datasets.CIFAR10(r"D:\深度学习\100_土堆数据集\dataset", train=True, transform=torchvision.transforms.ToTensor())      
# 语法：下载并加载 CIFAR10 测试集 (train=False)。
test_data = torchvision.datasets.CIFAR10(r"D:\深度学习\100_土堆数据集\dataset", train=False, transform=torchvision.transforms.ToTensor())      

# 语法：获取数据集长度。
train_data_size = len(train_data)
test_data_size = len(test_data)
print("训练数据集的长度：{}".format(train_data_size))
print("测试数据集的长度：{}".format(test_data_size))

# ----------------------
# 4. 加载数据 (DataLoader)
# ----------------------
# 语法：创建数据加载器。
# 作用：batch_size=64 表示一次打包 64 张图片喂给模型。DataLoader 会自动处理打乱数据(shuffle)和打包。
train_dataloader = DataLoader(train_data, batch_size=64)        
test_dataloader = DataLoader(test_data, batch_size=64)

# ----------------------
# 5. 初始化模型与配置
# ----------------------
# 实例化模型
tudui = Tudui() 
# 语法：将模型移动到 GPU 或 CPU。
# 注意：对于 nn.Module 模型对象，.to(device) 是原地操作(in-place)，不需要重新赋值，但写赋值也没错。
tudui = tudui.to(device) 

# 损失函数
# 语法：创建交叉熵损失函数，专门用于多分类任务。
loss_fn = nn.CrossEntropyLoss()
loss_fn = loss_fn.to(device) # 将损失函数也移动到设备上

# 优化器
learning = 0.01  # 学习率
# 语法：创建 SGD (随机梯度下降) 优化器。
# 参数：tudui.parameters() 告诉优化器要更新哪些参数（就是网络里的权重）。
optimizer = torch.optim.SGD(tudui.parameters(), learning)   

# ----------------------
# 6. 训练辅助变量
# ----------------------
total_train_step = 0 # 记录总共训练了多少步
total_test_step = 0  # 记录总共测试了多少轮
epoch = 30           # 训练轮数：把整个数据集学 30 遍


# ----------------------
# 7. 开始训练循环
# ----------------------
for i in range(epoch):
    print("-----第 {} 轮训练开始-----".format(i+1))
    
    # === 训练阶段 ===
    # 语法：将模型设置为训练模式。
    # 作用：启用 Dropout 和 BatchNorm 层（虽然这个简单模型里没有，但这是好习惯）。
    tudui.train() 
    
    for data in train_dataloader:
        imgs, targets = data            
        
        # 语法：将数据移动到设备 (GPU)。
        # 注意：对于 Tensor 数据，.to(device) 不是原地操作，必须把返回值重新赋给变量！
        imgs = imgs.to(device) 
        targets = targets.to(device)
        
        # 1. 前向传播：图片扔进网络，算出预测值
        outputs = tudui(imgs)
        
        # 2. 计算损失：看预测值和真实标签 targets 差多少
        loss = loss_fn(outputs, targets) 
        
        # 3. 反向传播与优化 (标准三部曲)
        optimizer.zero_grad()   # 梯度清零：清除上一步残留的梯度，否则会累加
        loss.backward()         # 反向传播：计算新的梯度
        optimizer.step()        # 更新参数：根据梯度调整权重
        
        total_train_step = total_train_step + 1
        
        # 每训练 100 次打印一次信息
        if total_train_step % 100 == 0:
            # .item() 将 Tensor 类型的 loss 转换为标准 Python 数字
            print("训练次数:{},Loss:{}".format(total_train_step, loss.item())) 
            
    
    # === 测试/验证阶段 ===
    # 语法：将模型设置为评估模式。
    # 作用：冻结 Dropout 和 BatchNorm，保证测试结果稳定。
    tudui.eval() 
    total_test_loss = 0
    total_accuracy = 0
    
    # 语法：停止梯度计算。
    # 作用：测试阶段不需要反向传播，关掉梯度可以极大节省显存并加速计算。
    with torch.no_grad(): 
        for data in test_dataloader:
            imgs, targets = data
            imgs = imgs.to(device)
            targets = targets.to(device)
            outputs = tudui(imgs)
            
            # 累加 Loss
            loss = loss_fn(outputs, targets)
            total_test_loss = total_test_loss + loss.item()
            
            # 计算正确率
            # outputs.argmax(1): 找出每一行（每张图）概率最大的那个类别的索引（即预测结果）
            # (predicted == targets): 比较预测是否正确，得到 True/False
            # .sum(): 将 True 视为 1 累加，得到这一批次猜对的数量
            accuracy = (outputs.argmax(1) == targets).sum()
            total_accuracy = total_accuracy + accuracy
            
    # 打印本轮测试结果
    print("整体测试集上的Loss:{}".format(total_test_loss))
    # 正确率 = 猜对的总数 / 测试集总数
    print("整体测试集上的正确率:{}".format(total_accuracy/test_data_size))
    
    total_test_step = total_test_step + 1
    
    # === 保存模型 ===
    # 语法：保存整个模型对象（结构+参数）。
    # 注意：更推荐用 state_dict 保存，但作为新手直接 save 整个模型最方便。
    torch.save(tudui, "D:\\深度学习\\100_土堆数据集\\model\\tudui_{}.pth".format(i)) 
    print("模型已保存")
    


训练数据集的长度：50000
测试数据集的长度：10000
-----第 1 轮训练开始-----
训练次数:100,Loss:2.282930374145508
训练次数:200,Loss:2.268775463104248
训练次数:300,Loss:2.2180774211883545
训练次数:400,Loss:2.087719440460205
训练次数:500,Loss:2.013606309890747
训练次数:600,Loss:1.978783130645752
训练次数:700,Loss:1.9969037771224976
整体测试集上的Loss:311.38084638118744
整体测试集上的正确率:0.28929999470710754
模型已保存
-----第 2 轮训练开始-----
训练次数:800,Loss:1.8477585315704346
训练次数:900,Loss:1.8340317010879517
训练次数:1000,Loss:1.8973135948181152
训练次数:1100,Loss:1.9406659603118896
训练次数:1200,Loss:1.663216471672058
训练次数:1300,Loss:1.6241077184677124


KeyboardInterrupt: 

# 2. 验证狗是否识别

① 完整的模型验证(测试，demo)套路，利用已经训练好的模型，然后给它提供输入。

In [7]:
import torchvision
from PIL import Image
from torch import nn
import torch

# ==========================================
# 1. 读取并预处理图片
# ==========================================
image_path = 'D:\\深度学习\\100_土堆数据集\\imgs\\dog.png'
image = Image.open(image_path) # 使用 PIL 库打开图片

# 【坑点1】格式转换
# PNG 图片通常是 4 通道 (RGBA，多了一个透明度通道)，但我们的模型只接收 3 通道 (RGB)。
#如果不加这行，进网络的第一层卷积就会报错：Given groups=1, weight of size [32, 3, 5, 5], expected input[1, 4, 32, 32]...
image = image.convert("RGB")  
print(image)

# 定义变换：必须和训练时的变换保持一致
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((32,32)), # 缩放到模型要求的尺寸 (32x32)
    torchvision.transforms.ToTensor()       # 转为 Tensor，并归一化到 [0,1]
])

image = transform(image)
print(image.shape) # 输出: torch.Size([3, 32, 32]) -> (通道, 高, 宽)

# ==========================================
# 2. 定义模型结构
# ==========================================
# ⚠️ 注意：使用 torch.load 加载整个模型时，
# 当前代码文件里必须有这个类的定义，否则程序不知道 "Tudui" 是个什么东西。
class Tudui(nn.Module):
    def __init__(self):
        super(Tudui, self).__init__()        
        self.model1 = nn.Sequential(
            nn.Conv2d(3,32,5,1,2),
            nn.MaxPool2d(2),
            nn.Conv2d(32,32,5,1,2),
            nn.MaxPool2d(2),
            nn.Conv2d(32,64,5,1,2),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64*4*4,64),
            nn.Linear(64,10)
        )
        
    def forward(self, x):
        x = self.model1(x)
        return x

# ==========================================
# 3. 加载模型并预测
# ==========================================

# 【坑点2】设备映射 map_location
# 如果你的模型是在 GPU (cuda) 上训练并保存的，但现在想在只有 CPU 的电脑上运行，
# 必须加上 map_location=torch.device('cpu')，否则会报错。
# 修改前：
# model = torch.load('D:\\...\\tudui_1.pth', map_location=torch.device('cpu'))

# 修改后 (加上 weights_only=False)：
model = torch.load('D:\\深度学习\\100_土堆数据集\\model\\tudui_1.pth', map_location=torch.device('cpu'), weights_only=False)

# 【坑点3】增加 Batch 维度
# 现在的 image 是 3维的 (3, 32, 32)。
# 但 PyTorch 模型要求的输入必须是 4维的 (Batch_Size, Channel, Height, Width)。
# 所以需要 reshape 变成 (1, 3, 32, 32)，那个 "1" 就是 Batch_Size。
image = torch.reshape(image,(1,3,32,32)) 

# 切换到测试模式 (虽然这个简单模型没影响，但这是好习惯)
model.eval()

# 开启无梯度模式 (省内存，不记录计算图)
with torch.no_grad(): 
    output = model(image)

# 打印原始输出 (10个数字，代表10个类别的分数)
print(output)

# 打印最终预测结果
# argmax(1) 表示在横向(第1维度)取最大值的索引
# 比如输出是 [-1.2, 3.5, 0.1...]，3.5 最大，索引是 1，则预测结果是类别 1 (dog)
print(output.argmax(1))

<PIL.Image.Image image mode=RGB size=307x173 at 0x20E1CEC8130>
torch.Size([3, 32, 32])
tensor([[-1.7995, -1.0654,  0.8879,  1.2731,  1.1375,  1.8303,  1.7666,  1.2773,
         -2.8012, -1.4486]])
tensor([5])


# 3. 验证飞机是否识别

In [8]:
import torchvision
from PIL import Image
from torch import nn
import torch

image_path = r'D:\深度学习\100_土堆数据集\imgs\plane.png'
image = Image.open(image_path) # PIL类型的Image
image = image.convert("RGB")  # 4通道的RGBA转为3通道的RGB图片
print(image)

transform = torchvision.transforms.Compose([torchvision.transforms.Resize((32,32)),   
                                            torchvision.transforms.ToTensor()])

image = transform(image)
print(image.shape)

class Tudui(nn.Module):
    def __init__(self):
        super(Tudui, self).__init__()        
        self.model1 = nn.Sequential(
            nn.Conv2d(3,32,5,1,2),
            nn.MaxPool2d(2),
            nn.Conv2d(32,32,5,1,2),
            nn.MaxPool2d(2),
            nn.Conv2d(32,64,5,1,2),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64*4*4,64),
            nn.Linear(64,10)
        )
        
    def forward(self, x):
        x = self.model1(x)
        return x

model = torch.load('D:\\深度学习\\100_土堆数据集\\model\\tudui_1.pth', map_location=torch.device('cpu'), weights_only=False) 
print(model)
image = torch.reshape(image,(1,3,32,32)) # 转为四维，符合网络输入需求
model.eval()
with torch.no_grad():  # 不进行梯度计算，减少内存计算
    output = model(image)
output = model(image)
print(output)
print(output.argmax(1)) # 概率最大类别的输出

<PIL.Image.Image image mode=RGB size=245x181 at 0x20E1D27CE50>
torch.Size([3, 32, 32])
Tudui(
  (model1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=1024, out_features=64, bias=True)
    (8): Linear(in_features=64, out_features=10, bias=True)
  )
)
tensor([[-3.1593,  1.0889, -0.1619,  1.5965, -0.2149,  1.8753,  3.4098, -0.1995,
         -2.3709,  0.8905]], grad_fn=<AddmmBackward0>)
tensor([6])
